In [1]:
from pyquil.quil import Program
from pyquil.gates import *
import pyquil.api as api
import numpy as np
from numpy.fft import ifft   # to check that final answer is correct

In [2]:
def state_prep(list_qubits, prog):
    '''
    Prepare the state corresponding to list of qubits, e.g. [1, 1, 0] -> |110>
    
    :param list list_qubits: list of qubits whose state is to be prepared
    :return Program: Program preparing the required qubit state
    '''
    n = len(list_qubits)
    for i, x in enumerate(list_qubits):
        q = n - 1 - i
        if x == 0:
            prog.inst(I(q))
        elif x == 1:
            prog.inst(X(q))
        else:
            raise ValueError("Input must consts of (integers) 0s and 1s")

def qft(n, prog):
    '''
    Computes the Quantum Fourier Transform of an n-qubit state

    :param int n: number of qubits
    :return Program: Modified Program that would carry out quantum fourier transform of the input n-qubit state
    '''
    # create QFT circuit
    q = n - 1
    while q >= 0:
        # apply Hadamard
        prog.inst(H(q))
        for i in range(2, q+2):
            # apply controlled phase rotations
            prog.inst(CPHASE(2*np.pi/(2**i), q, q-i+1))
        # decrement qubit index
        q -= 1
    # SWAP the order of qubits
    for j in range(n//2):
        prog.inst(SWAP(j, n-1-j))

In [3]:
# provide input n-qubit state (e.g. [1, 0, 1] for |110>)
list_qubs = [1, 1, 0, 0, 1]

In [4]:
qvm = api.QVMConnection()

# prepare Program
p = Program()
state_prep(list_qubs, p)
qft(len(list_qubs), p)

# obtain wavefunction
wavefunc = qvm.wavefunction(p)
# print (wavefunc.amplitudes)

# check against FFT
qub_vec = [0 for _ in range(2**(len(list_qubs)))]
lexicographic_index = 0
for i, x in enumerate(list_qubs[::-1]):
    lexicographic_index += x * (2**i)
qub_vec[lexicographic_index] = 1

# display the amplitudes
print ("Fourier transform of the vector |" + ''.join([str(i) for i in list_qubs]) + "> has coefficients: ", wavefunc.amplitudes)
print ("*" * 30)

# demonstrate that this equals (inverse) FFT of corresponding vector
if not (np.allclose(wavefunc.amplitudes, ifft(qub_vec, norm="ortho"))):
    raise AssertionError("BAD!")
else:
    print ("Which matches the classical Fourier transform for the array: ", qub_vec)

Fourier transform of the vector |11001> has coefficients:  [ 1.76776695e-01+0.j          3.44874224e-02-0.17337998j
 -1.63320371e-01-0.06764951j -9.82118698e-02+0.14698445j
  1.25000000e-01+0.125j       1.46984450e-01-0.09821187j
 -6.76495125e-02-0.16332037j -1.73379981e-01+0.03448742j
  1.08244507e-17+0.1767767j   1.73379981e-01+0.03448742j
  6.76495125e-02-0.16332037j -1.46984450e-01-0.09821187j
 -1.25000000e-01+0.125j       9.82118698e-02+0.14698445j
  1.63320371e-01-0.06764951j -3.44874224e-02-0.17337998j
 -1.76776695e-01+0.j         -3.44874224e-02+0.17337998j
  1.63320371e-01+0.06764951j  9.82118698e-02-0.14698445j
 -1.25000000e-01-0.125j      -1.46984450e-01+0.09821187j
  6.76495125e-02+0.16332037j  1.73379981e-01-0.03448742j
 -1.08244507e-17-0.1767767j  -1.73379981e-01-0.03448742j
 -6.76495125e-02+0.16332037j  1.46984450e-01+0.09821187j
  1.25000000e-01-0.125j      -9.82118698e-02-0.14698445j
 -1.63320371e-01+0.06764951j  3.44874224e-02+0.17337998j]
****************************